<a href="https://colab.research.google.com/github/ruka38/daily/blob/master/LLM8_4_Faiss%E3%82%92%E5%88%A9%E7%94%A8%E3%81%97%E3%81%9F%E6%9C%80%E8%BF%91%E5%82%8D%E6%8E%A2%E7%B4%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets faiss-cpu scipy transformers[ja,torch]

In [ ]:
from datasets import load_dataset
paragraph_dataset = load_dataset("llm-book/jawiki-paragraphs", split="train")

In [ ]:
print(paragraph_dataset)

In [ ]:
from pprint import pprint
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

In [ ]:
# 計算量を抑えるために最初の段落のみを使用する
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

In [ ]:
print(paragraph_dataset)

In [ ]:
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

In [ ]:
# トークナイザとモデルの準備
from transformers import AutoModel, AutoTokenizer
model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

In [ ]:
# 読み込んだエンコーダをGPUメモリに移動する
device = "cuda:0"
encoder = encoder.to(device)

In [ ]:
# モデルの埋め込みによる計算
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
  """SimCSEのモデルを用いてテキストの埋め込みを計算する"""
  # テキストにトークナイザを適用
  tokenized_texts = tokenizer(texts,
                              padding=True,
                              truncation=True,
                              max_length=128,
                              return_tensors="pt",
                              ).to(device)

  # トークナイズされたテキストをベクトルに変換
  with torch.inference_mode():
    with torch.cuda.amp.autocast():
        encoded_texts = encoder(
            **tokenized_texts
        ).last_hidden_state[:,0]

  # ベクトルをNumpyのarrayに変換
  emb = encoded_texts.cpu().numpy().astype(np.float32)
  # ベクトルのノルムが1になるように正規化
  emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)

  return emb

In [ ]:
# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
        },
    batched=True,
)

In [ ]:
print(paragraph_dataset)

In [ ]:
pprint(paragraph_dataset[0])

In [ ]:
# Faiss
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定してからのFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データのembeddingsフィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

In [ ]:
query_text = "日本語は、主に日本で話されている言語である"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings",
    embed_texts([query_text])[0],
    k=10
)

# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(f"score: {score:.3f}, title: {title}, text: {text}")